# 9. 类
[https://docs.python.org/zh-cn/3.8/tutorial/classes.html](https://docs.python.org/zh-cn/3.8/tutorial/classes.html)

类提供了一种组合数据和功能的方法。 

创建一个新类意味着创建一个新的`对象类型`，从而允许创建一个该类型的新`实例` 。 

类继承机制允许多个基类，派生类可以覆盖它基类的任何方法，一个方法可以调用基类中相同名称的的方法。

和模块一样，类也拥有 Python 天然的动态特性：它们`在运行时创建，可以在创建后修改`。

## 9.1. 名称和对象

多个名称（在多个作用域内）可以绑定到同一个对象。

## 9.2. Python 作用域和命名空间

【好好理解】

namespace （命名空间）是一个从名字到对象的映射。 

大部分命名空间当前都由 Python 字典实现，但一般情况下基本不会去关注它们（除了要面对性能问题时），
而且也有可能在将来更改。 

下面是几个命名空间的例子：

- 存放内置函数的集合（包含 abs() 这样的函数，和内建的异常等）；
- 模块中的全局名称；
- 函数调用中的局部名称。 
- 从某种意义上说，对象的属性集合也是一种命名空间的形式。

不同命名空间中的名称之间绝对没有关系；
例如，两个不同的模块都可以定义一个 maximize 函数而不会产生混淆 --- 模块的用户必须在其前面加上模块名称。

任何跟在一个点号之后的名称都称为 属性。例如，在表达式 z.real 中，real 是对象 z 的一个属性。

在模块的属性和模块中定义的全局名称之间正好存在一个直观的映射：它们共享相同的命名空间

存在一个例外。 
模块对象有一个秘密的只读属性 `__dict__`，它返回用于实现模块命名空间的字典；
`__dict__` 是属性但不是全局名称。 
显然，使用这个将违反命名空间实现的抽象，应当仅被用于事后调试器之类的场合。

在不同时刻创建的命名空间拥有不同的生存期。

- 包含内置名称的命名空间是在 Python 解释器启动时创建的，永远不会被删除。
- 模块的全局命名空间在【模块定义被读入时】创建；通常，模块命名空间也会持续到解释器退出。
- 被解释器的顶层调用执行的语句，从一个脚本文件读取或交互式地读取，被认为是 `__main__` 模块调用的一部分，因此它们拥有自己的全局命名空间。
- 函数的本地命名空间在调用该函数时创建，并在函数返回或抛出不在函数内部处理的错误时被删除。 当然，每次递归调用都会有自己的本地命名空间。

**一个 作用域 是一个命名空间可直接访问的 Python 程序的文本区域**。
这里的 “可直接访问” 意味着对名称的非限定引用会尝试在命名空间中查找名称。

虽然作用域是静态地确定的，但它们会被动态地使用:

- 最先搜索的最内部作用域包含局部名称  ①
- 从最近的封闭作用域开始搜索的任何封闭函数的作用域包含非局部名称，也包括非全局名称  ②
- 倒数第二个作用域包含当前模块的全局名称  ③
- 最外面的作用域（最后搜索）是包含内置名称的命名空间 ④

In [6]:
def outer_f():
    b = 2 # ②
    def inner_f():
        a = 1  # ①
        print(a)
        print(b)
        print(c)
    inner_f()

if __name__ == "__main__":
    c = 3  # ③
    outer_f()

1
2
3


如果一个名称被声明为全局变量，则所有引用和赋值将直接指向 包含该模块的全局名称 的中间作用域【下例中的 a 所在作用域】。

In [4]:
a = "python"
def f():
    global a
    a = "java"
    print("inner a = " +a)
    print(id(a))

f()
print("outer a = " +a)

inner a = java
2240553605744
outer a = java


要重新绑定在最内层作用域以外找到的变量，可以使用 nonlocal 语句声明为非本地变量。

如果没有被声明为非本地变量，这些变量将是只读的（尝试写入这样的变量只会在最内层作用域中创建一个 新的 局部变量，而同名的外部变量保持不变）。


In [9]:
def outer_f():
    a = "python"
    def inner_f():
        nonlocal a
        a = "java"
        print("inner a = " +a)
        print(id(a))

    inner_f()
    print("outer a = " +a)
    print(id(a))

outer_f()


inner a = java
2240553605744
outer a = java
2240553605744


In [ ]:
def outer_f():
    a = 1
    def inner_f():
        # nonlocal a  # 没有这条语句，会报错
        # a += 1
        print(a)
        print(id(a))

    inner_f()
    print("------------")

    print(a)
    print(id(a))

outer_f()


通常，当前局部作用域将（按字面文本）引用当前函数的局部名称。

在函数以外，局部作用域将引用与全局作用域相一致的命名空间：模块的命名空间。【？？】

类定义将在局部命名空间内再放置另一个命名空间。

In [10]:
a = "python"
def f():
    b = "java"
    print("a = " +a)
    print("b = " +b)

f()


a = python
b = java


重要的是应该意识到作用域是按字面文本来确定的：  

**在一个模块内定义的函数的全局作用域就是该模块的命名空间，无论该函数从什么地方或以什么别名被调用** 

另一方面，实际的名称搜索是在运行时动态完成的 --- 但是，Python 正在朝着“编译时静态名称解析”的方向发展，
因此不要过于依赖动态名称解析！ （事实上，局部变量已经是被静态确定了。）

如果不存在生效的 global 或 nonlocal 语句 -- 则对名称的赋值总是会进入最内层作用域。 【仅作用在最内层作用域】

赋值不会复制数据 --- 它们只是将名称绑定到对象。【重新绑定一个新的对象上】

In [1]:
a = "python"
def f():
    a = "java"
    print("inner a = " +a)
    print(id(a))

f()
print("outer a = " +a)
print(id(a))

inner a = java
3094991467824
outer a = python
3095033106352


删除也是如此：语句 `del x` 会从局部作用域所引用的命名空间中移除对 x 的绑定。 

事实上，所有引入新名称的操作都是使用局部作用域：特别地，import 语句和函数定义会在局部作用域中绑定模块或函数名称。

In [3]:
def outer_f():
    a = "python"
    def inner_f():
        a = "java"
        del a
        try:
            print("inner a = " +a)
        except NameError:
            print("inner")
    inner_f()

    try:
        print("outer a = " +a)
    except NameError:
        print("outer")
outer_f()

inner
outer a = python


In [5]:
def outer_f():
    a = "python"
    def inner_f():
        nonlocal a
        a = "java"
        del a
        try:
            print("inner a = " +a)
        except NameError:
            print("inner")
    inner_f()
    
    try:
        print("outer a = " +a)
    except NameError:
        print("outer")
    
outer_f()

inner
outer


global 和 nonlocal 总结：

- global 语句可被用来表明特定变量生存于全局作用域并且应当在其中被重新绑定；
- nonlocal 语句表明特定变量生存于外层作用域中并且应当在其中被重新绑定。



### 9.2.1. 作用域和命名空间示例

In [6]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()   # global 语句可被用来表明特定变量生存于全局作用域并且应当在其中被重新绑定
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)



After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam
